# Churn target optimisation

In [ ]:
from datetime import timedelta
import os

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from src.modeling import run_modeling
from config.config import THRESHOLD

In [ ]:
os.chdir("..")

## Loading relationship quality

In [ ]:
df_add = pd.read_csv("data/sales_client_relationship_dataset.csv")

In [ ]:
df_add.columns

In [ ]:
df_add["quali_relation"].value_counts()

## Loading model predictions

In [ ]:
train, test, clf = run_modeling(folds=1)

In [ ]:
# predict and add probabilites to test
preds = clf.predict_proba(test.drop(columns=["client_id", "churn"]))[:, 1]
test["preds"] = preds

## Target optimisation

In [ ]:
opt_df = test.merge(df_add, on="client_id", how="left")

In [ ]:
df = pd.read_csv("data/transactions_dataset.csv", sep=";")
df["date_order"] = pd.to_datetime(df["date_order"])

In [ ]:
test_stamp = df.date_order.max() - timedelta(days=THRESHOLD)
df = df[df.date_order < test_stamp]

In [ ]:
df = df[df.client_id.isin(opt_df.client_id.unique())]

In [ ]:
df = df[df.date_order >= (df.date_order.max() - timedelta(days=365))]

In [ ]:
sales_value = df.groupby("client_id").sales_net.sum().reset_index()
sales_value.columns = ["client_id", "sales_last_year"]

In [ ]:
opt_df = opt_df.merge(sales_value, on="client_id", how="left")

In [ ]:
plt.figure(figsize=(10, 5))
sns.scatterplot(
    data=opt_df, x="preds", y="sales_last_year", hue="quali_relation"
)

In [ ]:
plt.figure(figsize=(10, 5))
sns.scatterplot(
    data=opt_df[opt_df.preds > 0.2],
    x="quali_relation",
    y="sales_last_year",
    hue="preds",
)